## Downloading Dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/wav2vec2/challenge4/challenge_4_training_dataset.zip" -d "/content/train/"

Archive:  /content/drive/MyDrive/wav2vec2/challenge4/challenge_4_training_dataset.zip
   creating: /content/train/challenge_4_training_dataset/
   creating: /content/train/challenge_4_training_dataset/s2_train_release/
   creating: /content/train/challenge_4_training_dataset/s2_train_release/bird/
  inflating: /content/train/challenge_4_training_dataset/s2_train_release/bird/train_bird_8418.wav  
  inflating: /content/train/challenge_4_training_dataset/s2_train_release/bird/train_bird_8423.wav  
  inflating: /content/train/challenge_4_training_dataset/s2_train_release/bird/train_bird_8429.wav  
  inflating: /content/train/challenge_4_training_dataset/s2_train_release/bird/train_bird_8448.wav  
  inflating: /content/train/challenge_4_training_dataset/s2_train_release/bird/train_bird_8449.wav  
  inflating: /content/train/challenge_4_training_dataset/s2_train_release/bird/train_bird_8451.wav  
  inflating: /content/train/challenge_4_training_dataset/s2_train_release/bird/train_bird_8452.

In [ ]:
!pip install --upgrade transformers

     |████████████████████████████████| 2.5MB 7.8MB/s 
     |████████████████████████████████| 3.3MB 53.5MB/s 
     |████████████████████████████████| 901kB 51.4MB/s 


In [ ]:
!pip install s3fs
!pip install datasets

     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 122kB 17.9MB/s 
     |████████████████████████████████| 7.4MB 17.3MB/s 
     |████████████████████████████████| 1.3MB 49.9MB/s 
     |████████████████████████████████| 143kB 69.5MB/s 
     |████████████████████████████████| 143kB 67.0MB/s 
     |████████████████████████████████| 296kB 62.3MB/s 
  Created wheel for aiobotocore: filename=aiobotocore-1.3.1-cp37-none-any.whl size=46636 sha256=51342d3d7935fafb17a9e4c3d2b088566469ae9b5af4d7403deaffc7b9e33369
  Stored in directory: /root/.cache/pip/wheels/c9/6b/d6/8aba64c4d5011ca6a7a7eef469016a6e3ccd50c14b09ab44e1
Successfully built aiobotocore
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urlli

In [ ]:
!pip install torchaudio

     |████████████████████████████████| 1.9MB 9.0MB/s 


# Importing libraries

In [ ]:
import os
import sys
import numpy as np
import pandas as pd

# current torch version is 1.7.0+cu101
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchaudio
from torchaudio import transforms

from collections import namedtuple
import random

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import IPython.display as ipd

import librosa
import soundfile as sf

# check if cuda GPU is available, make sure you're using GPU runtime on Google Colab
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device) # you should output "cuda"

cuda


# Reading in audio files

In [ ]:
train_audio = '/content/train/challenge_4_training_dataset/s2_train_release'

data = []

# Read in all files inside each folder in the root folder

for folder in sorted(os.listdir(train_audio)):
    for file in sorted(os.listdir(train_audio+'/'+folder)):
        speech_array, sampling_rate = sf.read(train_audio+'/'+folder+'/'+file)
        data.append((file, speech_array, sampling_rate, folder))
        print(f'{file} appended to list \n')

# Create a dataframe for analysis/visualization/whatever

df = pd.DataFrame(data, columns=['file_name', 'speech_array', 'sampling_rate', 'label'])


Streaming output truncated to the last 5000 lines.
train_dog_6885.wav appended to list 

train_dog_6886.wav appended to list 

train_dog_6887.wav appended to list 

train_dog_6888.wav appended to list 

train_dog_6889.wav appended to list 

train_dog_6890.wav appended to list 

train_dog_6891.wav appended to list 

train_dog_6893.wav appended to list 

train_dog_6894.wav appended to list 

train_dog_6895.wav appended to list 

train_dog_6896.wav appended to list 

train_dog_6897.wav appended to list 

train_dog_6898.wav appended to list 

train_dog_6899.wav appended to list 

train_dog_6900.wav appended to list 

train_dog_6901.wav appended to list 

train_dog_6902.wav appended to list 

train_dog_6903.wav appended to list 

train_dog_6904.wav appended to list 

train_dog_6905.wav appended to list 

train_dog_6907.wav appended to list 

train_dog_6908.wav appended to list 

train_dog_6909.wav appended to list 

train_dog_6910.wav appended to list 

train_dog_6911.wav appended to list 


Looking at audio info

In [ ]:
df

,file_name,speech_array,sampling_rate,label
0,train_bird_8418.wav,"[-3.0517578125e-05, -0.000457763671875, -0.000...",16000,bird
1,train_bird_8423.wav,"[-0.001434326171875, -0.00152587890625, -0.001...",16000,bird
2,train_bird_8429.wav,"[0.000457763671875, 0.00115966796875, 0.001464...",16000,bird
3,train_bird_8448.wav,"[9.1552734375e-05, 3.0517578125e-05, 6.1035156...",16000,bird
4,train_bird_8449.wav,"[3.0517578125e-05, 6.103515625e-05, 3.05175781...",16000,bird
...,...,...,...,...
3345,train_zero_4499.wav,"[0.0, 6.103515625e-05, 0.0, 3.0517578125e-05, ...",16000,zero
3346,train_zero_4505.wav,"[9.1552734375e-05, 0.0, 6.103515625e-05, 3.051...",16000,zero
3347,train_zero_4520.wav,"[-9.1552734375e-05, -6.103515625e-05, -3.05175...",16000,zero
3348,train_zero_4535.wav,"[-0.000152587890625, -0.00030517578125, -0.000...",16000,zero


# Splitting audio files into train/val sets

In [ ]:
train, test = train_test_split(df, test_size=0.1, random_state=42, shuffle=True)

# Building the character vocab for the given training labels

In [ ]:
vocab = df.label.unique()

vocab_list = list({l for word in vocab for l in word})

vocab_dict = {k: v for v, k in enumerate(vocab_list)}

vocab_dict["<unk>"] = len(vocab_dict)
vocab_dict["<pad>"] = len(vocab_dict)

In [ ]:
vocab

array(['bird', 'cat', 'chicken', 'dog', 'down', 'eight', 'falcon', 'five',
       'four', 'go', 'left', 'nine', 'one', 'right', 'seven', 'six',
       'snake', 'stop', 'three', 'two', 'up', 'zero'], dtype=object)

To make sure that the model will not predict any character that is not in any of the words

In [ ]:
vocab_dict

{'<pad>': 23,
 '<unk>': 22,
 'a': 3,
 'b': 21,
 'c': 19,
 'd': 4,
 'e': 16,
 'f': 9,
 'g': 10,
 'h': 5,
 'i': 17,
 'k': 13,
 'l': 18,
 'n': 11,
 'o': 1,
 'p': 14,
 'r': 7,
 's': 15,
 't': 2,
 'u': 12,
 'v': 8,
 'w': 6,
 'x': 0,
 'z': 20}

In [ ]:
import json
with open('/content/drive/MyDrive/wav2vec2/challenge4/vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

# Converting the train/val sets into Datasets for input into the model later

In [ ]:
import s3fs
from datasets import Dataset

dataset_d = Dataset.from_pandas(train)
dataset_d2 = Dataset.from_pandas(test)

# Initializing the Wav2Vec2 model

In [ ]:
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
from transformers import Wav2Vec2ForCTC
from transformers import Wav2Vec2Config

tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/wav2vec2/challenge4/vocab.json", unk_token="<unk>", pad_token="<pad>", word_delimiter_token="|")

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

# Prepping the dataset for training/testing

In [ ]:
import itertools

In [ ]:
def prepare_dataset(batch):

    batch["input_values"] = processor(batch["speech_array"], sampling_rate=16000).input_values
    batch["input_values"] = list(itertools.chain.from_iterable(batch["input_values"]))
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["label"]).input_ids
    return batch

def prepare_dataset_test(batch):

    batch["input_values"] = processor(batch["speech_array"], sampling_rate=16000).input_values
    batch["input_values"] = list(itertools.chain.from_iterable(batch["input_values"]))
    
    return batch

In [ ]:
prep_d = dataset_d.map(prepare_dataset, remove_columns=dataset_d.column_names)

In [ ]:
prep_d2 = dataset_d2.map(prepare_dataset, remove_columns=dataset_d2.column_names)

Wav2Vec2's DataCollator

In [ ]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

# Using CER instead of WER as metrics because the labels are only 1 worded

In [ ]:
!pip3 install jiwer

     |████████████████████████████████| 51kB 4.4MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149822 sha256=2feb942d7855c2b9a92344fc7a8a39e5eae1996e9f69a0529d2b3176284b18ac
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


Change the "cer" to "wer" if using for phrases/sentences

In [ ]:

 
from datasets import load_metric

cer_metric = load_metric("cer") #"cer" https://github.com/huggingface/datasets/blob/master/metrics/cer/cer.py

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    cer = cer_metric.compute(predictions=pred_str, references=label_str) # cer

    return {"cer": cer}

# Loading the pre-trained models from previous

REQUIRED to load the pre-trained models from previous challenges here

In [ ]:
## Make sure that the following files are inside the local dir folder when importing

In [ ]:
import shutil 

%cp -av "/content/drive/MyDrive/wav2vec2/models_checkpoints/bestrun/" "/content/models/"

'/content/drive/MyDrive/wav2vec2/models_checkpoints/bestrun/' -> '/content/models/'
'/content/drive/MyDrive/wav2vec2/models_checkpoints/bestrun/special_tokens_map.json' -> '/content/models/special_tokens_map.json'
'/content/drive/MyDrive/wav2vec2/models_checkpoints/bestrun/preprocessor_config.json' -> '/content/models/preprocessor_config.json'
'/content/drive/MyDrive/wav2vec2/models_checkpoints/bestrun/tokenizer_config.json' -> '/content/models/tokenizer_config.json'
'/content/drive/MyDrive/wav2vec2/models_checkpoints/bestrun/vocab.json' -> '/content/models/vocab.json'
'/content/drive/MyDrive/wav2vec2/models_checkpoints/bestrun/config.json' -> '/content/models/config.json'
'/content/drive/MyDrive/wav2vec2/models_checkpoints/bestrun/pytorch_model.bin' -> '/content/models/pytorch_model.bin'


# Vocab/model file to take note of

Model trained from previous challenge is required to be loaded here

For this part you have to load in the vocab from the PREVIOUS challenge's vocab, because the test data had labels from the previous challenge that were not included in the current challenge.

You have to include the additional vocab into the previous one's as well and use it for the current challenge

I screwed up this part because I did not realize that until 2+ hours before submission deadline

Have to change the size of the output layer to match the total vocab size from previous+current

In [ ]:
import torch

pre_weight = torch.load('/content/models/pytorch_model.bin')

pre_weight['lm_head.weight'].resize_(24, 1024)
pre_weight['lm_head.bias'].resize_(24) # 24 classes = 24 chars

torch.save(pre_weight, "/content/models/pytorch_model.bin")

In [ ]:
model_n = "/content/models/"

# Setting the model's configs

Should increase dropout to prevent overfitting. 

Take note of the masked_time_length and mask_feature_length. If audio is too short, some important part of the audio may be masked

In [ ]:
model_config = Wav2Vec2Config.from_pretrained(model_n, 
                                              attention_dropout=0.12,
                                              hidden_dropout=0.12,
                                              feat_proj_dropout=0.0,
                                              mask_time_prob=0.001,
                                              mask_time_length=4,
                                              mask_feature_length=8,
                                              layerdrop=0.12,
                                              # gradient_checkpointing=True,
                                              apply_spec_augment=True,
                                              ctc_loss_reduction="mean",
                                              pad_token_id=processor.tokenizer.pad_token_id,
                                              vocab_size=len(processor.tokenizer)
                                              )

In [ ]:
model = Wav2Vec2ForCTC.from_pretrained(
    model_n,
    config=model_config
)

When the size of the output layer was trained, need to make the variables trainable

In [ ]:
for param in model.parameters():
  param.requires_grad=True

In [ ]:
model.lm_head.bias.shape

torch.Size([24])

Freeze the CNN feature extractor part of the model as we don't need to train it

In [ ]:
model.freeze_feature_extractor()

Parameters for training

Tune these to ensure that the Val Loss is as low as possible

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir='/content/checkpoints/',
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=50,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=0.00007,
  weight_decay=0.005,
  warmup_steps=500,
  save_total_limit=1,
  load_best_model_at_end=True
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=prep_d,
    eval_dataset=prep_d2,
    tokenizer=processor.feature_extractor,
)

In [ ]:
model.to("cuda")

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (4): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,)

# Training the model

Val Loss should be lower here

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Cer
500,31.059500,2.503101,0.444972
1000,2.858800,1.671827,0.146477
1500,1.482900,3.002808,0.065717
2000,0.944900,2.359112,0.050673
2500,0.703600,2.377048,0.045922
3000,0.525700,2.038140,0.039588
3500,0.401500,1.653514,0.034838
4000,0.354200,2.219903,0.042755
4500,0.318200,2.456583,0.039588


TrainOutput(global_step=4700, training_loss=4.120601671097126, metrics={'train_runtime': 5323.5754, 'train_samples_per_second': 28.317, 'train_steps_per_second': 0.883, 'total_flos': 4.56517317568896e+18, 'train_loss': 4.120601671097126, 'epoch': 49.99})

# Saving the trained model

In [ ]:
processor.save_pretrained('/content/drive/MyDrive/wav2vec2/challenge4/models_checkpoints/')

In [ ]:
model.save_pretrained('/content/drive/MyDrive/wav2vec2/challenge4/models_checkpoints/')

# Building an n-gram language model

In [ ]:
#### build kenlm model

In [ ]:
# !git clone https://github.com/ynop/py-ctc-decode
# %cd py-ctc-decode

Cloning into 'py-ctc-decode'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 28 (delta 6), reused 17 (delta 2), pack-reused 0
Unpacking objects: 100% (28/28), done.
/content/py-ctc-decode


In [ ]:
!pip install pypi-kenlm

In [ ]:
## new vocab
vocab_arr = vocab
# vocab_arr = ['bird', 'eight', 'falcon', 'five', 'four', 'nine', 'one', 'seven',
#        'six', 'snake', 'three', 'two', 'zero']

In [ ]:
vocab_arr

array(['bird', 'cat', 'chicken', 'dog', 'down', 'eight', 'falcon', 'five',
       'four', 'go', 'left', 'nine', 'one', 'right', 'seven', 'six',
       'snake', 'stop', 'three', 'two', 'up', 'zero'], dtype=object)

In [ ]:
vocab_arr = np.char.join(' ', vocab_arr)

In [ ]:
vocab_arr

array(['b i r d', 'c a t', 'c h i c k e n', 'd o g', 'd o w n',
       'e i g h t', 'f a l c o n', 'f i v e', 'f o u r', 'g o', 'l e f t',
       'n i n e', 'o n e', 'r i g h t', 's e v e n', 's i x', 's n a k e',
       's t o p', 't h r e e', 't w o', 'u p', 'z e r o'], dtype='<U13')

In [ ]:
with open('/content/drive/MyDrive/wav2vec2/challenge4/languagemodels_arpa/corpus.txt', 'w') as f:
    for item in vocab_arr:
        f.write("%s\n" % item)

In [ ]:
### install kenlm 

!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...
remote: Enumerating objects: 14047, done.
remote: Counting objects: 100% (360/360), done.
remote: Compressing objects: 100% (292/292), done.
remote: Total 14047 (delta 107), reused 121 (delta 55), pack-reused 13687
Receiving objects: 100% (14047/14047), 5.76 MiB | 17.70 MiB/s, done.
Resolving deltas: 100% (7987/7987), done.


In [ ]:
cd kenlm

/content/kenlm


In [ ]:
mkdir -p build

In [ ]:
!cmake . 

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Boost version: 

In [ ]:
!make -j 4

Scanning dependencies of target kenlm_util
[  1%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/bignum-dtoa.cc.o
[  2%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/bignum.cc.o
[  3%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/cached-powers.cc.o
[  5%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/diy-fp.cc.o
[  6%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/double-conversion.cc.o
[  7%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/fast-dtoa.cc.o
[  8%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/fixed-dtoa.cc.o
[ 10%] Building CXX object util/CMakeFiles/kenlm_util.dir/double-conversion/strtod.cc.o
[ 11%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/chain.cc.o
[ 12%] Building CXX object util/CMakeFiles/kenlm_util.dir/stream/count_records.cc.o
[ 13%] Building CXX object util/CMakeFiles/kenlm_util.dir/strea

Different n-grams models to be used for comparison later

In [ ]:
#### n grams 

In [ ]:
# n-gram, n=3
!./bin/lmplz --text /content/drive/MyDrive/wav2vec2/challenge4/languagemodels_arpa/corpus.txt --arpa /content/drive/MyDrive/wav2vec2/challenge4/languagemodels_arpa/word3.arpa --o 3 --skip_symbols --discount_fallback

=== 1/5 Counting and sorting n-grams ===
Reading /content/drive/MyDrive/wav2vec2/challenge4/languagemodels_arpa/corpus.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 5041512448 bytes == 0x557bda3e6000 @  0x7fc35cdd41e7 0x557bd8e407a2 0x557bd8ddb51e 0x557bd8dba2eb 0x557bd8da6066 0x7fc35af6dbf7 0x557bd8da7baa
tcmalloc: large alloc 16805036032 bytes == 0x557d06bdc000 @  0x7fc35cdd41e7 0x557bd8e407a2 0x557bd8e2f7ca 0x557bd8e30208 0x557bd8dba308 0x557bd8da6066 0x7fc35af6dbf7 0x557bd8da7baa
****************************************************************************************************
Unigram tokens 89 types 25
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:300 2:7607552000 3:14264160256
tcmalloc: large alloc 14264164352 bytes == 0x557bda3e6000 @  0x7fc35cdd41e7 0x557bd8e407a2 0x557bd8e2f7ca 0x557bd8e30208 0x557bd8dba8d7 0x557bd8da6066 0x7fc35af6dbf7 0x557bd8da7baa
tcmalloc: large alloc 7

In [ ]:
# n-gram, n=4
!./bin/lmplz --text /content/drive/MyDrive/wav2vec2/challenge4/languagemodels_arpa/corpus.txt --arpa /content/drive/MyDrive/wav2vec2/challenge4/languagemodels_arpa/words4.arpa --o 4 --skip_symbols --discount_fallback

=== 1/5 Counting and sorting n-grams ===
Reading /content/drive/MyDrive/wav2vec2/challenge4/languagemodels_arpa/corpus.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 4369309696 bytes == 0x55915eb42000 @  0x7fe9ae9221e7 0x55915d7527a2 0x55915d6ed51e 0x55915d6cc2eb 0x55915d6b8066 0x7fe9acabbbf7 0x55915d6b9baa
tcmalloc: large alloc 17477238784 bytes == 0x559263228000 @  0x7fe9ae9221e7 0x55915d7527a2 0x55915d7417ca 0x55915d742208 0x55915d6cc308 0x55915d6b8066 0x7fe9acabbbf7 0x55915d6b9baa
****************************************************************************************************
Unigram tokens 89 types 25
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:300 2:3722844416 3:6980333568 4:11168533504
tcmalloc: large alloc 11168538624 bytes == 0x55915eb42000 @  0x7fe9ae9221e7 0x55915d7527a2 0x55915d7417ca 0x55915d742208 0x55915d6cc8d7 0x55915d6b8066 0x7fe9acabbbf7 0x55915d6b9baa
tcmalloc: 

In [ ]:
# n-gram, n=5
!./bin/lmplz --text /content/drive/MyDrive/wav2vec2/challenge4/languagemodels_arpa/corpus.txt --arpa /content/drive/MyDrive/wav2vec2/challenge4/languagemodels_arpa/words5.arpa --o 5 --skip_symbols --discount_fallback

=== 1/5 Counting and sorting n-grams ===
Reading /content/drive/MyDrive/wav2vec2/challenge4/languagemodels_arpa/corpus.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 3855278080 bytes == 0x55b688eda000 @  0x7fcb894c01e7 0x55b687c457a2 0x55b687be051e 0x55b687bbf2eb 0x55b687bab066 0x7fcb87659bf7 0x55b687bacbaa
tcmalloc: large alloc 17991278592 bytes == 0x55b76eb88000 @  0x7fcb894c01e7 0x55b687c457a2 0x55b687c347ca 0x55b687c35208 0x55b687bbf308 0x55b687bab066 0x7fcb87659bf7 0x55b687bacbaa
****************************************************************************************************
Unigram tokens 89 types 25
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:300 2:2133825536 3:4000923136 4:6401476608 5:9335487488
tcmalloc: large alloc 9335488512 bytes == 0x55b688eda000 @  0x7fcb894c01e7 0x55b687c457a2 0x55b687c347ca 0x55b687c35208 0x55b687bbf8d7 0x55b687bab066 0x7fcb87659bf7 0x55b687bacbaa

In [ ]:
# n-gram, n=2
!./bin/lmplz --text /content/drive/MyDrive/wav2vec2/challenge4/languagemodels_arpa/corpus.txt --arpa /content/drive/MyDrive/wav2vec2/challenge4/languagemodels_arpa/words2.arpa --o 2 --skip_symbols --discount_fallback

=== 1/5 Counting and sorting n-grams ===
Reading /content/drive/MyDrive/wav2vec2/challenge4/languagemodels_arpa/corpus.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 5958156288 bytes == 0x5585f71a2000 @  0x7f009b0521e7 0x5585f5b777a2 0x5585f5b1251e 0x5585f5af12eb 0x5585f5add066 0x7f00991ebbf7 0x5585f5adebaa
tcmalloc: large alloc 15888400384 bytes == 0x55875a3c6000 @  0x7f009b0521e7 0x5585f5b777a2 0x5585f5b667ca 0x5585f5b67208 0x5585f5af1308 0x5585f5add066 0x7f00991ebbf7 0x5585f5adebaa
****************************************************************************************************
Unigram tokens 89 types 25
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:300 2:21871712665
tcmalloc: large alloc 21871714304 bytes == 0x558b0e4c2000 @  0x7f009b0521e7 0x5585f5b777a2 0x5585f5b667ca 0x5585f5b67208 0x5585f5af18d7 0x5585f5add066 0x7f00991ebbf7 0x5585f5adebaa
Substituting fallback discounts for 

In [ ]:
#####

In [ ]:
# load kenlm model
kenlm_model = "/content/drive/MyDrive/wav2vec2/challenge4/languagemodels_arpa/words4.arpa"

# Installing the CTC Beam Search Decoder to be used instead of Wav2Vec2's default decoder(greedy)

In [ ]:
!git clone https://github.com/ynop/py-ctc-decode
%cd py-ctc-decode

Cloning into 'py-ctc-decode'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 28 (delta 6), reused 17 (delta 2), pack-reused 0
Unpacking objects: 100% (28/28), done.
/content/kenlm/py-ctc-decode


In [ ]:
import ctcdecode

In [ ]:
asd = processor.tokenizer.get_vocab()
sorted_voc = sorted(asd.keys(), key=lambda k: asd[k])

In [ ]:
sorted_voc

['x',
 'o',
 't',
 'a',
 'd',
 'h',
 'w',
 'r',
 'v',
 'f',
 'g',
 'n',
 'u',
 'k',
 'p',
 's',
 'e',
 'i',
 'l',
 'c',
 'z',
 'b',
 '<unk>',
 '<pad>']

In [ ]:
voc = sorted_voc
voc = [x.replace('<pad>', '_') for x in voc]

In [ ]:
voc

['x',
 'o',
 't',
 'a',
 'd',
 'h',
 'w',
 'r',
 'v',
 'f',
 'g',
 'n',
 'u',
 'k',
 'p',
 's',
 'e',
 'i',
 'l',
 'c',
 'z',
 'b',
 '<unk>',
 '_']

In [ ]:
# prepare decoder and decode logits via shallow fusion
alpha = 0.5 # LM Weight
beta = 0.5 # LM Usage Reward

word_lm_scorer = ctcdecode.CharOfWordKenLMScorer(kenlm_model, alpha, beta) # use your own kenlm model

decoder = ctcdecode.BeamSearchDecoder(
    voc,
    num_workers=4,
    beam_width=100,
    scorers=[word_lm_scorer],
    cutoff_prob=np.log(0.000001),
    cutoff_top_n=40,

)

#text = decoder.decode_batch(tensorcpu)

# Prepping test data

In [ ]:
#### unzip test files

In [ ]:
!unzip "/content/drive/MyDrive/wav2vec2/challenge4/challenge_4_test_dataset.zip" -d "/content/test/"

Archive:  /content/drive/MyDrive/wav2vec2/challenge4/challenge_4_test_dataset.zip
   creating: /content/test/challenge_4_test_dataset/
  inflating: /content/test/challenge_4_test_dataset/challenge_4_test_0000.wav  
  inflating: /content/test/challenge_4_test_dataset/challenge_4_test_0002.wav  
  inflating: /content/test/challenge_4_test_dataset/challenge_4_test_0004.wav  
  inflating: /content/test/challenge_4_test_dataset/challenge_4_test_0005.wav  
  inflating: /content/test/challenge_4_test_dataset/challenge_4_test_0006.wav  
  inflating: /content/test/challenge_4_test_dataset/challenge_4_test_0008.wav  
  inflating: /content/test/challenge_4_test_dataset/challenge_4_test_0009.wav  
  inflating: /content/test/challenge_4_test_dataset/challenge_4_test_0010.wav  
  inflating: /content/test/challenge_4_test_dataset/challenge_4_test_0012.wav  
  inflating: /content/test/challenge_4_test_dataset/challenge_4_test_0014.wav  
  inflating: /content/test/challenge_4_test_dataset/challenge_4_t

In [ ]:
real_test = "/content/test/challenge_4_test_dataset"

test_data = []

for file in sorted(os.listdir(real_test)):

  speech_array, sampling_rate = sf.read(real_test+'/'+file)
  test_data.append((file, speech_array, sampling_rate))
  print(f'{file} appended to list \n')

test_df = pd.DataFrame(test_data, columns=['file_name', 'speech_array', 'sampling_rate'])

wa = Dataset.from_pandas(test_df)


was = wa.map(prepare_dataset_test, remove_columns=wa.column_names)

challenge_4_test_0000.wav appended to list 

challenge_4_test_0001.wav appended to list 

challenge_4_test_0002.wav appended to list 

challenge_4_test_0004.wav appended to list 

challenge_4_test_0005.wav appended to list 

challenge_4_test_0006.wav appended to list 

challenge_4_test_0007.wav appended to list 

challenge_4_test_0008.wav appended to list 

challenge_4_test_0009.wav appended to list 

challenge_4_test_0010.wav appended to list 

challenge_4_test_0012.wav appended to list 

challenge_4_test_0014.wav appended to list 

challenge_4_test_0015.wav appended to list 

challenge_4_test_0016.wav appended to list 

challenge_4_test_0017.wav appended to list 

challenge_4_test_0018.wav appended to list 

challenge_4_test_0019.wav appended to list 

challenge_4_test_0020.wav appended to list 

challenge_4_test_0021.wav appended to list 

challenge_4_test_0022.wav appended to list 

challenge_4_test_0023.wav appended to list 

challenge_4_test_0024.wav appended to list 

challenge_

In [ ]:
if torch.cuda.is_available():
    model.cuda()

# Inference on the test data

Commented part is for CTCBeammSearchDecoder + Language Model

In [ ]:
import tqdm

final_pred = []
for i in range(was.shape[0]):    
    input_dict = processor(was[i]["input_values"], sampling_rate=16000, return_tensors="pt", padding=True)

    with torch.no_grad():
      logits = model(input_dict.input_values.to("cuda")).logits

    # lm beam search decoder
    # tensorcpu = logits.cpu()

    # text = decoder.decode(tensorcpu[0])
    # print(f'{i} {text}')

    pred_ids = torch.argmax(logits, dim=-1)[0]
    prediction = processor.decode(pred_ids)
    final_pred.append(prediction)

    # final_pred.append(text)

In [ ]:
test_df["labels"] = final_pred

In [ ]:
test_df.drop(columns=['speech_array', 'sampling_rate'], inplace=True)

In [ ]:
test_df['labels'][0]

'bird'

# Submission

In [ ]:
test_df.to_csv('/content/drive/MyDrive/wav2vec2/challenge4_NOLM.csv', index=False, header=False)